<a href="https://www.kaggle.com/code/youlikecode/predicting-stock-pyspark?scriptVersionId=122712559" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Problem definition

In this project, we’ll learn how to predict stock prices using python, pandas, and scikit-learn. 
we’ll create a machine learning model, and develop a back-testing engine. 

In this case, let’s say that we are trading stocks. We’re interested in making profitable stock trades with minimal risk. So when we buy a stock, we want to be fairly certain that the price will increase. We’ll buy stock when the market opens, and sell it when the market closes.

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 13.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=edf27d1638174abb1a0e1d8985941a938431393493ee589931fa57ff0e4cb141
  Stored in directory: /root/.cache/pip/wheels/5a/54/9b/a89cac960efb57c4c35d41cc7c9f7b80daa21108bc376339b7
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
# Loading the packages
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
# Create an instance of your Spark session
spark = SparkSession.builder.appName('stock_price_prediction').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/20 01:31:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Load the data from the csv file
path = "/kaggle/input/daily-historical-stock-prices-1970-2018/historical_stock_prices.csv"
df = spark.read.csv(path, header=True, inferSchema=True)
df = df.withColumn('date', df['date'].cast('timestamp'))

In [5]:
# Pre-process the data
data = df.select('open', 'close', 'low', 'high', 'volume').na.drop()
data = data.select(*(data[c].cast("float").alias(c) for c in data.columns))

In [6]:
# Split the data into a training set and a test set
(trainingData, testData) = data.randomSplit([0.7, 0.3], seed=42)

In [7]:
# Define the characteristics and the target variable
assembler = VectorAssembler(inputCols=['open', 'low', 'high', 'volume'], outputCol='features')
trainingData = assembler.transform(trainingData)
testData = assembler.transform(testData)
trainingData = trainingData.select("features", "close")
testData = testData.select("features", "close")

In [8]:
# Create the model using PySpark's linear regression algorithm
lr = LinearRegression(labelCol="close", featuresCol="features")

In [9]:
# Train the model using the training set
lrModel = lr.fit(trainingData)

23/03/20 01:32:02 WARN Instrumentation: [d172f338] regParam is zero, which might cause numerical instability and overfitting.


In [10]:
# Perform the prediction using the test suite
predictions = lrModel.transform(testData)

In [11]:
# Evaluate the model using evaluation metrics

# mean square error
evaluator = RegressionEvaluator(labelCol="close", predictionCol="prediction", metricName="mse")
mse = evaluator.evaluate(predictions)

# coefficient of determination
evaluator = RegressionEvaluator(labelCol="close", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)

print('MSE:', mse)
print('R2:', r2)

MSE: 7474.812815990718
R2: 0.9990425522046996
